In [8]:
import bw2data as bd
from bw2data.backends.schema import ActivityDataset as AD

In [2]:
bd.projects.set_current('GSA for archetypes')

In [3]:
if "Swiss residual electricity mix" in bd.databases:
    del bd.databases["Swiss residual electricity mix"]

In [13]:
sr = bd.Database("Swiss residual electricity mix")
sr.register()

Create `ActivityDataset` as this is the only way to specify the `id`.

In [10]:
AD.create(
    id=100000,
    code="CH-residual",
    database="Swiss residual electricity mix",
    location="CH", 
    name="Swiss residual electricity mix", 
    product="electricity, high voltage",
    type="process",
    data=dict(
        unit="kilowatt_hour",
        comment="Difference between generation fractions for SwissGrid and ENTSO",
        location="CH", 
        name="Swiss residual electricity mix", 
        reference_product="electricity, high voltage",
    )
)   

<ActivityDataset: 100000>

In [17]:
act = bd.get_activity(id=100000)
act

'Swiss residual electricity mix' (kilowatt_hour, CH, None)

In [18]:
act.new_exchange(input=act, type="production", amount=1).save()

In [27]:
switzerland_residual = {
    'electricity production, hydro, reservoir, alpine region': 0.2814150228066876,
    'electricity production, hydro, run-of-river': 0.636056236216345,
    'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014': 0.012048389472504549,
    'heat and power co-generation, biogas, gas engine': 0.059773867534434144,
    'heat and power co-generation, natural gas, 500kW electrical, lean burn': 0.006612375072688834,
    'electricity production, wind, >3MW turbine, onshore': 0.0010024269784498687,
    'electricity production, wind, 1-3MW turbine, onshore': 0.0026554668750543753,
    'electricity production, wind, <1MW turbine, onshore': 0.00043621504383564323
}

In [28]:
act_mapping = {
    act: switzerland_residual[act['name']]
    for act in bd.Database("ecoinvent 3.8 cutoff") 
    if act['location'] == 'CH'
    and act['unit'] == 'kilowatt hour'
    and act['name'] in switzerland_residual
}
assert len(act_mapping) == len(switzerland_residual)
act_mapping

{'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, CH, None): 0.012048389472504549,
 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, CH, None): 0.2814150228066876,
 'electricity production, hydro, run-of-river' (kilowatt hour, CH, None): 0.636056236216345,
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, CH, None): 0.00043621504383564323,
 'heat and power co-generation, natural gas, 500kW electrical, lean burn' (kilowatt hour, CH, None): 0.006612375072688834,
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, CH, None): 0.0026554668750543753,
 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, CH, None): 0.0010024269784498687,
 'heat and power co-generation, biogas, gas engine' (kilowatt hour, CH, None): 0.059773867534434144}

In [29]:
for key, value in act_mapping.items():
    act.new_exchange(input=key, type='technosphere', amount=value).save()

In [30]:
for exc in act.exchanges():
    print(exc)

Exchange: 1 kilowatt_hour 'Swiss residual electricity mix' (kilowatt_hour, CH, None) to 'Swiss residual electricity mix' (kilowatt_hour, CH, None)>
Exchange: 0.012048389472504549 kilowatt hour 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, CH, None) to 'Swiss residual electricity mix' (kilowatt_hour, CH, None)>
Exchange: 0.2814150228066876 kilowatt hour 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, CH, None) to 'Swiss residual electricity mix' (kilowatt_hour, CH, None)>
Exchange: 0.636056236216345 kilowatt hour 'electricity production, hydro, run-of-river' (kilowatt hour, CH, None) to 'Swiss residual electricity mix' (kilowatt_hour, CH, None)>
Exchange: 0.00043621504383564323 kilowatt hour 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, CH, None) to 'Swiss residual electricity mix' (kilowatt_hour, CH, None)>
Exchange: 0.006612375072688834 kilowatt hour 'heat and power co-generation, natural ga

In [31]:
sr.process()